In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
# mpl.use("pdf")
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU serif"
mpl.rcParams["mathtext.it"] = "CMU serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
from scipy.optimize import curve_fit
import scipy
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
# import PlotLib.Stats as Stats
import PlotLib.Hist1D as Hist1D
import PlotLib.Hist2D as Hist2D
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

import time

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)

import ROOT

def get_mask_seedPix(data):
    return data[:,:,M["Charge"]] == arr([np.max(data[:,:,M["Charge"]], axis=1) for i in range(4)]).T

In [ ]:
nEvents = 4*50000
# nEvents = 4*200
runIDs = arr([184,190,192])
runIDs = arr([190])
filepaths = {
    184:f"/home/jona/DESY/analysis_TB/output/csv/184_thr200.csv",
    190:f"/home/jona/DESY/analysis_TB/output/csv/190_thr200.csv",
    192:f"/home/jona/DESY/analysis_TB/output/csv/190_thr520.csv",
}

runs = {}
for runID in runIDs:
    runs[runID] = load_run(runID, nEvents*4, filepaths[runID])
    # runs[runID]["data"] has the following structure:
    # [ event , pixel, entry ] where entry is the index given by the map M
    
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],nEvents)
    runs[runID]["threshold"] = 200 if runID in [184,190] else 520
    runs[runID]["MTP"] = 5
    
M = runs[runIDs[0]]["M"]

def findMax(func, par, x0=10):
    def f(x, par):
        return -1 * func(x, par)
    f2 = np.vectorize(f, excluded=[1])
    return scipy.optimize.fmin(f2, x0, args=(par,), disp=False)[0]

print(M)

In [ ]:
FileName = "map_inpix/Hitmap"

runID = 190
thr = 200

zRange = arr([5,25])
binN = arr([14,10])
binRange = arr([[-0.5,0.5], [-0.5,0.5]])

hist = Hist2D.Hist_2D(binN, binRange)

mask = runs[runID]["data"][:,:,M["Charge"]] > thr
mask = np.logical_and(mask, get_mask_seedPix(runs[runID]["data"]))

xs = runs[runID]["data"][:,:,M["TrkInterceptCol"]].flatten()[mask.flatten()] - runs[runID]["data"][:,:,M["PixCol"]].flatten()[mask.flatten()]
ys = runs[runID]["data"][:,:,M["TrkInterceptRow"]].flatten()[mask.flatten()] - runs[runID]["data"][:,:,M["PixRow"]].flatten()[mask.flatten()]
hist.fill(xs, ys)

fig, ax = Plot.create_fig(figsize=(5,7))
label = f"Entries ({int(hist.getIntegral(False)/1000)}k tot.)"
hist.draw(ax, aspect=5/7, overflow=False, cbar_label=label, ax_cbar="auto")

Plot.finalize(runs[runID], fig, ax, 
    title=r"Inpix Trk Intercepts with Charge > 200 $e$",
    xlabel=r"pixel column",
    ylabel= r"pixel row",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":False, "fontsize":8, "thr":thr, "edgeCut":None, "minTrkPlanes":None},
    grid=False, legend_loc="upper right")
Plot.savefig(fig, FileName)

In [ ]:
FileName = "map_inpix/Risetime"
runID = 190
thr = 200

binN = arr([14,10])
binN = arr([11,9])
binRange = arr([[-0.5,0.5], [-0.5,0.5]])
Bootstrap_N = 10

mask = runs[runID]["data"][:,:,M["Charge"]] > thr
mask = np.logical_and(mask, get_mask_seedPix(runs[runID]["data"]))

runs[runID]["data"][:,:,M["TrkInterceptCol"]].flatten()[mask.flatten()] - runs[runID]["data"][:,:,M["PixCol"]].flatten()[mask.flatten()]

xs = runs[runID]["data"][:,:,M["TrkInterceptCol"]].flatten()[mask.flatten()] - runs[runID]["data"][:,:,M["PixCol"]].flatten()[mask.flatten()]
ys = runs[runID]["data"][:,:,M["TrkInterceptRow"]].flatten()[mask.flatten()] - runs[runID]["data"][:,:,M["PixRow"]].flatten()[mask.flatten()]
zs = runs[runID]["data"][:,:,M["Risetime"]].flatten()[mask.flatten()]

hist = Hist2D.Plot_2D(binN, binRange, statistic="MedianBootstrap", Bootstrap_N=Bootstrap_N)
hist.fillData(xs, ys, zs)

fig, ax = Plot.create_fig(figsize=(5,7))
label = f"Median Seed Pixel Risetime (20%-80%) / ns"
hist.draw(ax, aspect=5/7, ax_cbar="auto", overflow=False, cbar_label=label)
Plot.finalize(runs[runID], fig, ax, 
    title="Inpixel Risetime",
    xlabel=r"pixel column / pitch ($35\,\mu$m)",
    ylabel= r"pixel row / pitch ($25\,\mu$m)",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":False, "fontsize":8, "thr":thr, "edgeCut":None, "minTrkPlanes":None},
    grid=False, legend_loc="upper right")
Plot.savefig(fig, FileName)

In [ ]:
FileName = "map_inpix/ToA"
runID = 190
thr = 200

binN = arr([14,10])
binN = arr([11,9])
binRange = arr([[-0.5,0.5], [-0.5,0.5]])
Bootstrap_N = 10

mask = runs[runID]["data"][:,:,M["Charge"]] > thr
mask = np.logical_and(mask, get_mask_seedPix(runs[runID]["data"]))

runs[runID]["data"][:,:,M["TrkInterceptCol"]].flatten()[mask.flatten()] - runs[runID]["data"][:,:,M["PixCol"]].flatten()[mask.flatten()]

xs = runs[runID]["data"][:,:,M["TrkInterceptCol"]].flatten()[mask.flatten()] - runs[runID]["data"][:,:,M["PixCol"]].flatten()[mask.flatten()]
ys = runs[runID]["data"][:,:,M["TrkInterceptRow"]].flatten()[mask.flatten()] - runs[runID]["data"][:,:,M["PixRow"]].flatten()[mask.flatten()]
zs = runs[runID]["data"][:,:,M["ToA"]].flatten()[mask.flatten()]

hist = Hist2D.Plot_2D(binN, binRange, statistic="MedianBootstrap", Bootstrap_N=Bootstrap_N)
hist.fillData(xs, ys, zs)

fig, ax = Plot.create_fig(figsize=(5,7))
label = f"Median Seed Pixel ToA (50% CFD) / ns"
hist.draw(ax, aspect=5/7, ax_cbar="auto", overflow=False, cbar_label=label)
Plot.finalize(runs[runID], fig, ax, 
    title="Inpixel Time of Arrival",
    xlabel=r"pixel column / pitch ($35\,\mu$m)",
    ylabel= r"pixel row / pitch ($25\,\mu$m)",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":False, "fontsize":8, "thr":thr, "edgeCut":None, "minTrkPlanes":None},
    grid=False, legend_loc="upper right")
Plot.savefig(fig, FileName)